In [1]:
import pandas as pd
import numpy as np
import regex as re
import yfinance as yf
import pickle
import os
import datetime
import seaborn as sns

## Correlation of Fidelity Funds
*Notebook by Peter Amerkhanian, 3/14/2022*

This notebook is an attempt to look at how different low cost Fidelity funds covary. Please note that this notebook is soley for education purposes and is not financial advice.
#### Data:
- `fidelity_funds.xlsx` 92 Fidelity funds that meet the following criteria:
    - No Transaction Fees, No minimum investment
    - below 0.75% net expense ratio
- `yfinance` API call that gathers historic returns for each fund

#### Question:
A typical investing strategy is to seek assets that are [countercyclical](https://www.investopedia.com/terms/c/countercyclicalstock.asp#:~:text=Counter%2Dcyclical%20stocks%20refer%20to,cyclical%20stocks%20will%20do%20well.) so as to mitigate portfolio risk during economic downturns. Given this, I'm curious which Fidelity funds are the most countercyclical, or, which are most inversely correlated with the returns of the stock market.

#### Methods:
I take the daily historical returns (in percent change) of a given group of Fidelity funds and calculate the correlation coefficient, $\rho$ between each fund and a baseline fund, `"FXAIX"`, which is a fund that seeks to mimic the S&P500.

For improved accuracy, I compute 95% confidence intervals of each $\rho_{fund, FXAIX}$ via bootstrapping

### Data Processing - `fidelity_funds.xlsx`

In [2]:
funds_df_raw = pd.read_excel("fidelity_funds.xlsx")

In [3]:
funds_df_raw.head()

,Name,Morningstar Category,YTD (Daily),1 Yr,3 Yr,5 Yr,10 Yr,Life of Fund,Expense Ratio - Net,Expense Ratio - Gross,Morningstar- Overall,Morningstar- 3yrs,Morningstar- 5yrs,Morningstar- 10yrs
0,Fidelity ZERO<sup>SM</sup> Extended Market Ind...,Mid-Cap Blend,-0.1035,2.67% (02/28/2022),12.84% (02/28/2022),-,-,9.45% (02/28/2022),0.00000,0.00000,2 (360 Rated),2 (360 Rated),-,-
1,Fidelity ZERO<sup>SM</sup> Large Cap Index Fun...,Large Blend,-0.1227,14.41% (02/28/2022),18.37% (02/28/2022),-,-,14.65% (02/28/2022),0.00000,0.00000,4 (1232 Rated),4 (1232 Rated),-,-
2,Fidelity ZERO<sup>SM</sup> Total Market Index ...,Large Blend,-0.1204,12.64% (02/28/2022),17.59% (02/28/2022),-,-,14.36% (02/28/2022),0.00000,0.00000,3 (1232 Rated),3 (1232 Rated),-,-
3,Fidelity 500 Index Fund (FXAIX),Large Blend,-0.1153,16.37% (02/28/2022),18.23% (02/28/2022),15.16% (02/28/2022),14.58% (02/28/2022),10.89% (02/28/2022),0.00015,0.00015,5 (1232 Rated),4 (1232 Rated),4 (1108 Rated),5 (819 Rated)
4,Fidelity Total Market Index Fund (FSKAX),Large Blend,-0.1220,11.94% (02/28/2022),17.43% (02/28/2022),14.60% (02/28/2022),14.20% (02/28/2022),8.63% (02/28/2022),0.00015,0.00015,4 (1232 Rated),3 (1232 Rated),3 (1108 Rated),4 (819 Rated)


In [4]:
# Extract ticker symbols from the "Name" field
funds_df_raw["Ticker"] = (funds_df_raw["Name"]
                          .str.extract(r"\(([A-Z]*)\)")
                          .astype(str))

In [5]:
# Drop any row without performance data from this year
funds_df = funds_df_raw.dropna(subset=["YTD (Daily)"])

### Data Retrieval from `yfinance`
This script returns comprehensive historical data for each fund then pickles that data. If the data has already been retrieved, the script will display the date/time of the last retrieval

In [6]:
if os.path.exists('fidelity_funds.pickle'):
    print("Fidelity fund info already retrieved @", 
          datetime.datetime.fromtimestamp(os.path.getctime('fidelity_funds.pickle')))
else:
    stocks = {}
    for ticker in funds_df["Ticker"]:
        stock = yf.Ticker(ticker)
        # get all market data
        hist = stock.history(period="max")
        stocks[ticker] = hist
        with open('fidelity_funds.pickle', 'wb') as f:
            pickle.dump(stocks, f)

Fidelity fund info already retrieved @ 2022-03-14 16:59:09.785354


### Processing `yfinance` data 

In [7]:
with open('fidelity_funds.pickle', 'rb') as f:
    stocks = pickle.load(f)

In [8]:
for stock_name, stock_data in stocks.items():
    stock_data['Pct_Change'] = stock_data['Close'].pct_change()

In [9]:
t = pd.concat(
    [df[["Pct_Change"]].rename(columns={"Pct_Change": name})
     for name, df in stocks.items()], 
    axis=1)

In [10]:
t_no_na = t.dropna(thresh=30)
t_no_na.corr()["FXAIX"].sort_values().to_frame().style.background_gradient(cmap='coolwarm')

,FXAIX
FGOVX,-0.392708
FUAMX,-0.390454
FNBGX,-0.389334
FSTGX,-0.367870
FUMBX,-0.333870
FFXSX,-0.284356
FXNAX,-0.280808
FTHRX,-0.220852
FNSOX,-0.203848
FNDSX,-0.198729


In [11]:
baseline = "FXAIX"
num_iter = 1000
bootstrap_results = pd.concat(
    [(
        t_no_na.sample(100, replace=True)
        .corr()[baseline]
        .to_frame()
        .sort_values(by=baseline)
        .rename(columns={baseline: rep})
    )
     for rep in range(num_iter)], axis=1).T

In [12]:
bottom_covariates = bootstrap_results.quantile(q=[0.05, .5, 0.95]).T.sort_values(by=.5, ascending=True).head(20)

In [13]:
(
    pd.merge(bottom_covariates,
             funds_df[["Morningstar Category",
                       "Ticker",
                       "Life of Fund",
                       "Expense Ratio - Net"]],
             left_index=True, right_on="Ticker"
            )
    .reset_index().drop("index", axis=1))

,0.05,0.5,0.95,Morningstar Category,Ticker,Life of Fund,Expense Ratio - Net
0,-0.661198,-0.392124,-0.002671,Intermediate Government,FGOVX,6.52% (02/28/2022),0.00450
1,-0.731197,-0.375290,0.146908,Intermediate Government,FUAMX,4.08% (02/28/2022),0.00030
2,-0.748433,-0.366202,0.181585,Long Government,FNBGX,5.86% (02/28/2022),0.00030
3,-0.634020,-0.360908,-0.039962,Intermediate Government,FSTGX,4.67% (02/28/2022),0.00450
4,-0.688264,-0.332905,0.191391,Short Government,FUMBX,2.36% (02/28/2022),0.00030
5,-0.584918,-0.312342,0.138916,Intermediate Core Bond,FXNAX,5.57% (02/28/2022),0.00025
6,-1.000000,-0.311548,1.000000,Intermediate Core-Plus Bond,FFEBX,-3.29% (02/28/2022),0.00450
7,-0.559638,-0.289648,0.022296,Short Government,FFXSX,4.36% (02/28/2022),0.00450
8,-0.548684,-0.272182,0.224211,Intermediate Core Bond,FTHRX,6.74% (02/28/2022),0.00450
9,-0.666166,-0.256273,0.345657,Short-Term Bond,FNSOX,1.75% (02/28/2022),0.00030
